<h2>Initialize SparkSQL Application - Create SQL Context</h2>

In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

properties = {
    'username': 'postgres',
    'password': '20020202',
    'url': "jdbc:postgresql://localhost:5432/postgres",
    'table': 'fifa.player_data',
    'driver': 'org.postgresql.Driver'
}

def write_to_pgadmin(df, mode='overwrite'):
    df.write.format('jdbc').mode(mode)\
        .option("url", properties['url'])\
        .option("dbtable", properties['table'])\
        .option("user", properties['username'])\
        .option("password", properties['password'])\
        .option("Driver", properties['driver'])\
        .save()

def read_from_pgadmin():
    return spark.read.format("jdbc")\
        .option("url", properties['url'])\
        .option("dbtable", properties['table'])\
        .option("user", properties['username'])\
        .option("password", properties['password'])\
        .option("Driver", properties['driver'])\
        .load()

appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .set('spark.jars.packages', 'org.postgresql:postgresql:42.7.0')\
    .setAppName(appName)\
    .setMaster(master)

# conf = pyspark.SparkConf().\
#     set('spark.jars.packages', 'org.postgresql:postgresql:42.7.0')\
#     .setAppName(appName).setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder.getOrCreate()

C:\spark\spark-3.5.3-bin-hadoop3\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Task-I: Build and populate necessary tables

### Load Dataset from .csv files

In [2]:
import os
from pyspark.sql.functions import lit, monotonically_increasing_id
root = 'data'
soccer_data = spark.read.csv(os.path.join(root, 'players_15.csv'), header=True, inferSchema=True)
schema = soccer_data.schema
soccer_data = soccer_data.withColumn('year', lit(2015))
soccer_data = soccer_data.withColumn('gender', lit('male'))
for file in sorted(os.listdir(root)):
    if file.split('_')[-1][:-4] != '15':
        df = spark.read.csv(os.path.join(root, file), header=True, schema=schema)
        df = df.withColumn('year', lit(2000+int(file.split('_')[-1][:-4])))      # add new column for the year
        if file.startswith('player'):
            df = df.withColumn('gender', lit('Male'))
        else:
            df = df.withColumn('gender', lit('Female'))
        soccer_data = soccer_data.union(df)

soccer_data = soccer_data.withColumn('unique_id', monotonically_increasing_id())    # add unique id to each row
soccer_data.show(5)
soccer_data.printSchema()

+---------+--------------------+-----------------+--------------------+----------------+-------+---------+---------+--------+---+----------+---------+---------+------------+-------------------+--------------------+------------+-------------+------------------+----------------+-----------+-------------------------+--------------+----------------+--------------+---------------+--------------------+--------------+---------+-----------+------------------------+-------------+----------------+---------+------------------+--------------------+--------------------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-

### Push data to Postgres Database

In [3]:
write_to_pgadmin(soccer_data)

## Task-II: Conduct analytics on the dataset

In Year X, what were the Y clubs that had the highest number of players with 
contracts ending in year Z (or after)? 
- X is a year between (2015 and 2022, inclusively). 
- Y is a positive integer. 
- Z is a year that can hold the value of 2023 or a year after it.

In [ ]:
def get_club_based_on_contract_ending(df, X, Y, Z):
    assert type(X)==int and X >= 2015 and X <= 2022, 'X should be a year between (2014 and 2022, inclusively)'
    assert type(Y)==int and Y > 0, 'Y should be a positive integer'
    assert type(Z)==int and Z >= 2023, 'Z should be 2023 or a year after it'
    df_valid = df[(df['year']==X) & (df['club_contract_valid_until']==Z)]
    df_num_valid_per_club = df_valid.groupby('club_name').count().orderBy('count', ascending=False)
    top_Y_club = df_num_valid_per_club.select('club_name').limit(Y).rdd.flatMap(lambda x: x).collect()
    return top_Y_club

In [5]:
df = read_from_pgadmin()
df = df[df['gender']=='Male']
X = 2020
Y = 3
Z = 2025
print(get_club_based_on_contract_ending(df, X, Y, Z))

['Real Madrid CF', 'Real Sociedad', 'RCD Espanyol de Barcelona']


List the X clubs with the highest (or lowest) average player age for a given 
year Y.  
- X represents a positive integer, but you should handle a scenario if 
X is not positive value. 
- Y represents a year between 2015 and 2022 inclusively.  
- Provide the user with the ability to choose if they want the highest 
average age or the lowest average age.  
- Make sure to handle this scenario as well: if the user requests 5 
clubs with highest averages but there are 3 clubs that share the 
same count at rank number 5, please include all of them in your 
output 

In [6]:
def get_club_based_on_age(df, X, Y, mode):
    assert mode=='highest' or mode=='lowest', 'invalid mode'
    assert type(X)==int and X > 0, 'X shoul be a positive integer'
    assert type(Y)==int and Y >= 2015 and Y <= 2022, 'Y should be a year between 2014 and 2022 inclusively'
    df_valid = df[df['year']==Y]
    df_avg_age_per_club = df_valid.select(['club_name', 'age']).groupby('club_name').mean()
    df_unique_age = df_avg_age_per_club.select('avg(age)').distinct()
    if mode == 'highest':
        largest_Xth_age = df_unique_age.orderBy('avg(age)', ascending=False).collect()[X-1]['avg(age)']
        return df_avg_age_per_club[df_avg_age_per_club['avg(age)'] >= largest_Xth_age].select('club_name').rdd.flatMap(lambda x:x).collect()
    else:
        smallest_Xth_age = df_unique_age.orderBy('avg(age)').collect()[X-1]['avg(age)']
        return df_avg_age_per_club[df_avg_age_per_club['avg(age)'] <= smallest_Xth_age].select('club_name').rdd.flatMap(lambda x:x).collect()
    

In [7]:
df = read_from_pgadmin()
df = df[df['gender']=='Male']
X = 7
Y = 2020
print(get_club_based_on_age(df, X, Y, mode='highest'))

['Club Athletico Paranaense', 'Fortaleza', 'Santos', 'Avaí FC', 'Botafogo', 'Goiás', 'Internacional', 'Fluminense', 'Cruzeiro', 'Bahia', 'Grêmio', 'Clube Atlético Mineiro', 'Ceará Sporting Club', 'Associação Chapecoense de Futebol']


What is the most popular nationality in the dataset for each year? (i.e. display the 
most frequent nation for 2015, 2016, etc.). 

In [8]:
def get_most_popular_nationality_per_year(df):
    unique_years = df.select('year').distinct().rdd.flatMap(lambda x:x).collect()
    for year in sorted(unique_years):
        df_nationality_count = df[df['year']==year].groupby('nationality_name').count()
        most_popular_nationality = df_nationality_count.orderBy('count', ascending=False).collect()[0]['nationality_name']
        print(f'The most popular nationality in year {year} is {most_popular_nationality}')

In [9]:
df = read_from_pgadmin()
df[df['gender']=='Male']
get_most_popular_nationality_per_year(df)

The most popular nationality in year 2015 is England
The most popular nationality in year 2016 is England
The most popular nationality in year 2017 is England
The most popular nationality in year 2018 is England
The most popular nationality in year 2019 is England
The most popular nationality in year 2020 is England
The most popular nationality in year 2021 is England
The most popular nationality in year 2022 is England
